In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import pprint
import os
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

In [2]:
filepath = os.path.join('Resources','each_state_new_cancer_rate.csv')
df = pd.read_csv(filepath)
df = df.rename(columns = {'Unnamed: 0':'States'})
df.head()

,States,Cancer Type,Age,Sex,Race,Type,Age-Adjusted Rate,Case Count,Population
0,Nevada,All Types of Cancer,NaN,Male and Female,All Races and Ethnicities,Incidence,Data Suppressed,Data Suppressed,Data Suppressed
1,New Mexico,All Types of Cancer,NaN,Male and Female,All Races and Ethnicities,Incidence,350.4,9202,2092741
2,Arizona,All Types of Cancer,NaN,Male and Female,All Races and Ethnicities,Incidence,368.1,32863,7158024
3,Colorado,All Types of Cancer,NaN,Male and Female,All Races and Ethnicities,Incidence,387.2,24550,5691287
4,California,All Types of Cancer,NaN,Male and Female,All Races and Ethnicities,Incidence,388.4,171748,39461588


In [ ]:
url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&q='

states = df['States']
lat = []
lng = []
state_name = []

for state in states:
    query_url = url + state
    response = requests.get(query_url).json()
    state_name.append(response['name'])
    lat.append(response['coord']['lat'])
    lng.append(response['coord']['lon'])
new_df = pd.DataFrame({'Lat':lat,'Lon':lng,'States':state_name})
new_df.head()

In [ ]:
new_df = new_df.replace('State of Wyoming','Wyoming') 
new_df = new_df.replace('State of Maine','Maine')
new_df.head()

In [ ]:
combined_df = pd.merge(df, new_df, on = 'States', how = 'outer')
combined_df = combined_df [['States','Age-Adjusted Rate','Case Count','Population','Lat','Lon']]
combined_df = combined_df.iloc[1:]
combined_df ['Age-Adjusted Rate'] = combined_df ['Age-Adjusted Rate'].astype(float)
combined_df ['Lat'] = combined_df ['Lat'].astype(float)
combined_df ['Lon'] = combined_df ['Lon'].astype(float)
combined_df = combined_df.rename(columns={'Age-Adjusted Rate':'Rate','Case Count':'Cases'})
combined_df.head()

In [ ]:
plt.scatter(combined_df['Lat'], combined_df['Rate'])

In [ ]:
combined_df.boxplot('Rate', by='Lat', figsize=(10, 8))

In [ ]:
bins = [0,20,30,40,50,60]
names = ['0-19','20-29','30-39','40-49','50-59']
combined_df['Latbin'] = pd.cut(combined_df['Rate'], bins, labels=names)
combined_df(head)

# combined_df.boxplot('Rate', by='Lat', figsize=(10, 8))

In [ ]:
combined_df.hist(column='Rate')

In [ ]:
latitude2 = combined_df[combined_df['Lat'] == 38.3004]
latitude2.Rate.mean()

In [ ]:
stats.ttest_ind(latitude1.Rate, latitude2.Rate, equal_var=False)